In [ ]:
# sage documentation for SQLite interface
# http://doc.sagemath.org/html/en/reference/misc/sage/databases/sql_db.html

In [1]:
import time
load("kraus_tables.sage")
kraus = kraus_table_Q2
# CDB = CremonaDatabase(); CDB
p = 2
# note: the quadratic extensions of Q2 are Q2(sqrt(d)) with d = 3,5,7,2,6,10,14.

objfile = os.path.join('./', '2-ktype_mtype_data.sobj')
label_type_tuples = load(objfile)  # format: (cremona label (string), Kodaira type (Sage custom type), integer)

objfile = os.path.join('./', '2-ktype_mtype_first_examples.sobj')
first_examples = load(objfile)

In [10]:
def ctame(S):
    if S == KodairaSymbol('In*'):
        return 1/2
    elif S == KodairaSymbol('II'):
        return 1/6
    elif S == KodairaSymbol('II*'):
        return 5/6
    elif S == KodairaSymbol('III'):
        return 1/4
    elif S == KodairaSymbol('III*'):
        return 3/4
    elif S == KodairaSymbol('IV'):
        return 1/3
    elif S == KodairaSymbol('IV*'):
        return 2/3
    else:
        raise ValueError("Kodaira type In not implemented")

In [267]:
from collections import OrderedDict
table_skeleton = OrderedDict([
    ('label',{'sql':'TEXT', 'index':True, 'primary_key':True}), # the Cremona label
    ('mtype',{'sql':'TEXT', 'index':True,}), # monodromy type                         
    ('ktype',{'sql':'TEXT', 'index':True,}), # Kodaira type
    ('c',{'sql':'TEXT', 'index':True,}), # the Chai conductor
    ('c_tame',{'sql':'TEXT', 'index':True,}), # the tame part of the chai conductor
    ('c_wild',{'sql':'TEXT', 'index':True,}),# the wild part of the chai conductor
])
D = SQLDatabase()
D.create_table('twpg', table_skeleton) # twpg = totally wild potentially good
# D.show('twpg')

In [268]:
t0 = time.time()
for (label, symbol, mtype) in label_type_tuples:
    if mtype in [2,4,8]:
        E = EllipticCurve(label)
        LD = E.local_data(2)

        c = LD.discriminant_valuation()/12       
        ct = ctame(symbol)
        cw = c - ct
        
        D.add_row('twpg', (label, mtype, symbol, c, ct, cw))
        # print((label, c, ct, cw, mtype))
# D.show('twpg')
t1 = time.time()
print('Time: {}'.format(t1-t0))
# 194.962843895s on my personal laptop on 5 Oct. 2019

# don't run this unless you have recomputed everything 
# D.save('/home/cody/Dropbox/sage-notebooks/unstable-ECs/2-totally-wild.db')

Time: 191.098484039


In [271]:
# which chai conductors occur?
Q = SQLQuery(D, {'table_name':'twpg',
                 'display_cols':['c'], 
                 'expression':['c','>','0']})
Qout = Q.query_results(); 
Qout = set([str(l[0]) for l in Qout]); Qout

{'1', '1/2', '3/2', '3/4', '5/4'}

In [292]:
# conductors and types when M = C_2?
# which chai conductors occur?
Q = SQLQuery(D, {'table_name':'twpg',
                 'display_cols':['ktype','c', 'c_tame', 'c_wild'], 
                 'expression':['mtype','=','2']})
Qout = set(Q.query_results()); 
Qout = [map(lambda x: str(x), t) for t in Qout]
Qout

[['II', '1/2', '1/6', '1/3'],
 ['II*', '1', '5/6', '1/6'],
 ['I4*', '1', '1/2', '1/2'],
 ['I8*', '3/2', '1/2', '1']]

In [290]:
# conductors and types when M = C_4?
# which chai conductors occur?
Q = SQLQuery(D, {'table_name':'twpg',
                 'display_cols':['ktype','c', 'c_tame', 'c_wild'], 
                 'expression':['mtype','=','4']})
Qout = set(Q.query_results()); 
Qout = [map(lambda x: str(x), t) for t in Qout]
Qout

[['III*', '5/4', '3/4', '1/2'], ['III', '3/4', '1/4', '1/2']]

In [291]:
# conductors and types when M = Q_8?
# which chai conductors occur?
Q = SQLQuery(D, {'table_name':'twpg',
                 'display_cols':['ktype','c', 'c_tame', 'c_wild'], 
                 'expression':['mtype','=','8']})
Qout = set(Q.query_results()); 
Qout = [map(lambda x: str(x), t) for t in Qout]
Qout

[['I0*', '3/4', '1/2', '1/4'],
 ['III*', '5/4', '3/4', '1/2'],
 ['III', '3/4', '1/4', '1/2'],
 ['I3*', '1', '1/2', '1/2'],
 ['I2*', '1', '1/2', '1/2'],
 ['III', '1/2', '1/4', '1/4'],
 ['II', '1/2', '1/6', '1/3'],
 ['III*', '1', '3/4', '1/4'],
 ['I5*', '5/4', '1/2', '3/4']]